# Train SALLY ensemble

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

from madminer.sampling import SampleAugmenter
from madminer.sampling import multiple_benchmark_thetas
from madminer.sampling import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas
from madminer.ml import MLForge, EnsembleForge

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')

In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma/'

## Settings

In [4]:
n_estimators = 10

## Training function

In [5]:
def train_ensemble(filename, use_tight_cuts=True, n_estimators=10, **kwargs):
    cut_label = '_tight' if use_tight_cuts else ''
    
    ensemble = EnsembleForge(n_estimators, debug=False)

    ensemble.train_all(
        method='sally',
        x_filename=[sample_dir + 'train_local{}/x_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        t_xz0_filename=[sample_dir + 'train_local{}/t_xz_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        **kwargs
    )

    ensemble.calculate_expectation(
        x_filename=sample_dir + 'validation{}/x_validation.npy'.format(cut_label)
    )

    ensemble.save(model_dir + 'sally_ensemble_' + filename)

## All observables

In [7]:
train_ensemble('all', use_tight_cuts=False)

19:15  Training 10 estimators in ensemble
19:15  Training estimator 1 / 10 in ensemble
19:15  Starting training
19:15    Method:                 sally
19:15    Training data: x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/x_train_0.npy
19:15                   t_xz (theta0) at  /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/t_xz_train_0.npy
19:15    Features:               all
19:15    Method:                 sally
19:15    Hidden layers:          (100, 100)
19:15    Activation function:    tanh
19:15    Batch size:             128
19:15    Trainer:                amsgrad
19:15    Epochs:                 50
19:15    Learning rate:          0.001 initially, decaying to 0.0001
19:15    Validation split:       0.25
19:15    Early stopping:         True
19:15    Scale inputs:           True
19:15  Loading training data
19:15  Found 1000000 samples with 2 parameters and 26 observables
19:15  Rescalin

20:39  Training model
20:42    Epoch 5: train loss 24.57 ([24.57243346]), validation loss 44.46 ([44.46122286])
20:45    Epoch 10: train loss 24.50 ([24.50174225]), validation loss 44.42 ([44.41926205]) (*)
20:48    Epoch 15: train loss 24.34 ([24.33919337]), validation loss 44.46 ([44.46456015])
20:51    Epoch 20: train loss 24.07 ([24.06517427]), validation loss 44.65 ([44.65279397])
20:54    Epoch 25: train loss 23.69 ([23.69379805]), validation loss 44.78 ([44.7771647])
20:56    Epoch 30: train loss 23.34 ([23.33539544]), validation loss 44.98 ([44.9761648])
20:57  No improvement for 20 epochs, stopping training
20:57  Early stopping after epoch 11, with loss 44.41 compared to final loss 44.99
20:57  Finished training
20:57  Training estimator 6 / 10 in ensemble
20:57  Starting training
20:57    Method:                 sally
20:57    Training data: x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/x_train_5.npy
20:57                   t

22:11    Activation function:    tanh
22:11    Batch size:             128
22:11    Trainer:                amsgrad
22:11    Epochs:                 50
22:11    Learning rate:          0.001 initially, decaying to 0.0001
22:11    Validation split:       0.25
22:11    Early stopping:         True
22:11    Scale inputs:           True
22:11  Loading training data
22:11  Found 1000000 samples with 2 parameters and 26 observables
22:11  Rescaling inputs
22:11  Creating model for method sally
22:11  Training model
22:14    Epoch 5: train loss 55.74 ([55.74330641]), validation loss 29.26 ([29.25619013])
22:17    Epoch 10: train loss 55.57 ([55.56626205]), validation loss 29.21 ([29.21490525])
22:20    Epoch 15: train loss 55.22 ([55.22291917]), validation loss 29.27 ([29.26824157])
22:23    Epoch 20: train loss 54.82 ([54.8232569]), validation loss 29.36 ([29.35828795])
22:25    Epoch 25: train loss 54.45 ([54.45019626]), validation loss 29.42 ([29.423762])
22:28  No improvement for 20 epoch

In [ ]:
train_ensemble('all_tight', use_tight_cuts=True)

In [ ]:
train_ensemble(
    'all_tight_sgd',
    use_tight_cuts=True,
    trainer='sgd',
    nesterov_momentum=0.9,
    initial_lr=0.1,
    final_lr=0.01
)

In [ ]:
train_ensemble('resurrection_tight', use_tight_cuts=True)